In [ ]:
import geopandas as gpd
import pandas as pd
import osmnx as ox

In [ ]:
# city = 'borsele'
# city = 'breda'
city = 'alphen-waddinxveen'

In [ ]:
# boundaries = ['Borsele, Zeeland, Netherlands']
# boundaries = ['Breda, North Brabant, Netherlands']
boundaries = [
    'Alphen aan den Rijn, Zuid-Holland, Netherlands',
    'Waddinxveen, Zuid-Holland, Netherlands',
    'Boskoop, Zuid-Holland, Netherlands'
] 

In [ ]:
osm_data = gpd.read_file(f"/Users/cmartens/Documents/thesis_cf_martens/2.risk_analysis/output/{city}/osm_data_with_risk_{city}.geojson")

In [ ]:
no_fly_zones = gpd.read_file("/Users/cmartens/Documents/thesis_cf_martens/3.no_fly_zones/input/gdf_no_fly_zones_nl_27feb.geojson")

In [ ]:
no_fly_zones['air_type'].unique()

In [ ]:
counts = no_fly_zones.groupby(no_fly_zones.geometry.type)['area_type'].value_counts()

print("Aantal area_types per geometry type voor risk='no_fly_zone':")
print(counts)

In [ ]:
boundary = ox.geocode_to_gdf(boundaries)

# 2. Zorg dat beide in hetzelfde CRS staan
no_fly_zones = no_fly_zones.to_crs(boundary.crs)

# 3. Filter no-fly zones die binnen Breda vallen
# (Gebruik 'intersects' als je ook gedeeltelijke overlap wilt toestaan)
no_fly_in_boundarie = no_fly_zones[no_fly_zones.geometry.intersects(boundary.geometry.iloc[0])].copy()

In [ ]:
no_fly_in_boundarie['risk'] = 'no_fly_zone'

In [ ]:
concatenated_gdf = pd.concat([osm_data, no_fly_in_boundarie], ignore_index=True)

In [ ]:
# # 1. Projecteer eerst naar meters (bijv. RD New - EPSG:28992)
# concatenated_gdf = concatenated_gdf.to_crs(epsg=28992)

# # 2. Buffer points (cirkels van 50 meter)
# mask_points = (concatenated_gdf['risk'] == 'no_fly_zone') & (concatenated_gdf.geometry.type == 'Point')

# concatenated_gdf.loc[mask_points, 'geometry'] = concatenated_gdf.loc[mask_points, 'geometry'].apply(lambda pt: pt.buffer(50))

# # 3. Buffer linestrings (smalle polygonen van 5 meter breed)
# mask_lines = (concatenated_gdf['risk'] == 'no_fly_zone') & (concatenated_gdf.geometry.type == 'LineString')
# concatenated_gdf.loc[mask_lines, 'geometry'] = concatenated_gdf.loc[mask_lines, 'geometry'].apply(lambda ln: ln.buffer(2.5))

# # (Optioneel) Terugprojecteren naar WGS 84 (graden) als je export naar GeoJSON wilt
# concatenated_gdf = concatenated_gdf.to_crs(epsg=4326)


In [ ]:
concatenated_gdf[concatenated_gdf['risk'] == 'no_fly_zone']

In [ ]:
mask = concatenated_gdf['risk'] == 'no_fly_zone'
subset = concatenated_gdf[mask]

counts = subset.groupby(subset.geometry.type)['area_type'].value_counts()

print("Aantal area_types per geometry type voor risk='no_fly_zone':")
print(counts)

In [ ]:
concatenated_gdf[concatenated_gdf['area_type'] == 'Pedestrian and cycling paths']

In [ ]:
concatenated_gdf.to_file(f"/Users/cmartens/Documents/thesis_cf_martens/3.no_fly_zones/output/data_for_graph_{city}.geojson", driver="GeoJSON")